In [9]:
import pandas as pd
import numpy as np
import pickle
import math
from gams import *

Charging_station = [1,1,1,1]
trip_chains = [[4,13,18,29,34,45,50,61,66,76,82,92],
               [13,29,45,61,76,92]
               [10,21,26,37,42,53,58,69,74,84,90,99],
               [1,2,8,11,19,24,31,36,43,48,55,60,67,72,79,85,91,96,102,106,110,113],
               [3,5,12,16,23,28,35,40,47,52,59,64,71,77,83,88,95,100,105,109,112,115],
               [6,7,15,20,27,32,39,44,51,56,63,68,75,80,87,93,98,103,108,111,114,116],
               [9,14,25,30,41,46,57,62,73,78,89,94,104,107],
               [17,22,33,38,49,54,65,70,81,86,97,101]]
chrom_batt = [380,380,380,380,380,380,380]
fleet_size = 7

cs = 4

trip_information = pd.read_csv("Trip_information_NumericalStudy.csv")
terminal_distance = pd.read_csv("Between_terminal_distance_NumericalStudy.csv")
##terminals
#1---University Medical Center
#2---Salt Lake Central Station
#3---This is the Place State Park
#4---North Temple Station

From_depot_to_trip_time = pd.read_csv("From_depot_to_trip_time")
From_trip_to_depot_time = pd.read_csv("From_trip_to_depot_time")
From_depot_to_trip_energy = pd.read_csv("From_depot_to_trip_energy")
From_trip_to_depot_energy = pd.read_csv("From_trip_to_depot_energy")

available_charging_time = pd.read_csv("Between_trip_time_NumericalStudy.csv")
energy_need = pd.read_csv("Between_trip_energy_NumericalStudy.csv")

following_set = pickle.load(open('Following_set_dict.pickle','rb'))

########import the parameters of the chromosome to gams
# store trip chains to a dictionary
dict_trip_set = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    for j in trip_set:
        dict_trip_set[str(i),str(j)] = 1
#print(dict_trip_set)

# store the first trip of each bus to a dictionary
dict_first_trip = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    dict_first_trip[str(i), str(trip_set[0])] = 1
#print(dict_first_trip)

# store the last trip of each bus to a dictionary
dict_last_trip = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    dict_last_trip[str(i), str(trip_set[-1])] = 1
#print(dict_last_trip)

# store the adjacent trips of each bus to a dictionary
dict_adjacent_trips = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    for j in range(1, len(trip_set)):
        a = trip_set[j-1]
        b = trip_set[j]
        dict_adjacent_trips[str(i),str(a),str(b)] = 1
#print(dict_adjacent_trips)

# store the available time periods for charging before each trip of each bus to a dictionary
dict_available_charging_time = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    for j in range(1, len(trip_set)):
        trip1 = trip_set[j-1]
        trip2 = trip_set[j]
        trip1_endtime = trip_information.iloc[trip1-1]["End_time"]
        trip2_starttime = trip_information.iloc[trip2-1]["Start_time"]
        trip1_endterminal = trip_information.iloc[trip1-1]["End_terminal"]
        trip2_startterminal = trip_information.iloc[trip2-1]["Start_terminal"]
        between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
        between_time_need = math.ceil(between_distance/20*60)
        for l in range(int(trip1_endtime+between_time_need), int(trip2_starttime)):
            dict_available_charging_time[str(i),str(trip2),str(int(trip2_startterminal)),str(l)] = 1
#print(dict_available_charging_time)

# store the available time periods for charging at the depot for each bus
dict_available_charging_time_d = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    last_trip = trip_set[-1]
    first_trip = trip_set[0]
    last_trip_end_terminal = trip_information.iloc[last_trip-1]["End_terminal"]
    last_trip_end_time = trip_information.iloc[last_trip-1]["End_time"]
    first_trip_start_terminal = trip_information.iloc[first_trip-1]["Start_terminal"]
    first_trip_start_time = trip_information.iloc[first_trip-1]["Start_time"]
    arrive_depot_time = last_trip_end_time + math.ceil(From_trip_to_depot_time.iloc[last_trip-1][1]*60)
    leave_depot_time = first_trip_start_time - math.ceil(From_depot_to_trip_time.iloc[first_trip-1][1]*60)
    for t in range(int(arrive_depot_time),1441):
        dict_available_charging_time_d[str(i),str(t)] =1
    for t in range(1,int(leave_depot_time)):
        dict_available_charging_time_d[str(i),str(t)] =1
#print(dict_available_charging_time_d)

# store the start terminal of trip i to a dictionary
dict_start_terminal = {}
for i in range(116):
    start_terminal = trip_information.iloc[i]["Start_terminal"]
    dict_start_terminal[str(i+1), str(int(start_terminal))] = 1
#print(dict_start_terminal)

# store the end terminal of trip i to a dictionary
dict_end_terminal = {}
for i in range(116):
    end_terminal = trip_information.iloc[i]["End_terminal"]
    dict_end_terminal[str(i+1), str(int(end_terminal))] = 1
#print(dict_end_terminal)

# store the energy consumption of serving a trip of each bus to a dictionary
dict_energy_trip = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    for j in trip_set:
        trip_energy_need = trip_information.iloc[j-1]["Energy_need"]
        dict_energy_trip[str(i),str(j)] = trip_energy_need
#print(dict_energy_trip)

# store the energy consumption of serving two adjacent trips of each bus to a dictionary
dict_energy_between_trips = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    for j in range(1, len(trip_set)):
        trip1 = trip_set[j-1]
        trip2 = trip_set[j]
        trip1_endterminal = trip_information.iloc[trip1-1]["End_terminal"]
        trip2_startterminal = trip_information.iloc[trip2-1]["Start_terminal"]
        between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
        between_energy_need = between_distance*2.69
        dict_energy_between_trips[str(i),str(trip1),str(trip2)] = between_energy_need
#print(dict_energy_between_trips)

# store the energy consumption from the depot to the first trip of each bus to a dictionary
dict_energy_depot_to_first_trip = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    first_trip = trip_set[0]
    dict_energy_depot_to_first_trip[str(i),str(first_trip)] = From_depot_to_trip_energy.iloc[first_trip-1][1]
#print(dict_energy_depot_to_first_trip)

# store the energy consumption from the last trip to the depot of each bus to a dictionary
dict_energy_last_trip_to_depot = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    last_trip = trip_set[-1]
#    print(last_trip)
    dict_energy_last_trip_to_depot[str(i),str(last_trip)] = From_trip_to_depot_energy.iloc[last_trip-1][1]
#print(dict_energy_last_trip_to_depot)

# store the total operating time per day per bus
dict_operating_time = {}
for i in range(1,fleet_size+1):
    trip_set = trip_chains[i-1]
    first_trip = trip_set[0]
    last_trip = trip_set[-1]
    time1 = From_depot_to_trip_time.iloc[first_trip-1][1]
    first_trip_start_time = trip_information.iloc[first_trip-1]["Start_time"]
    last_trip_end_time = trip_information.iloc[last_trip-1]["End_time"]
    time2 = last_trip_end_time - first_trip_start_time
    time3 = From_trip_to_depot_time.iloc[last_trip-1][1]
    dict_operating_time[str(i)] = time1*60 + time2 + time3*60
#print(dict_operating_time)  

# store whether a terminal is equipped with a charging station
dict_term_charging = {}
for i in range(cs):
    dict_term_charging[str(i+1)] = Charging_station[i]
#print(dict_term_charging)

ws = GamsWorkspace(system_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/")
# create new GamsDatabase instance
db = ws.add_database()

fleet_size_set = []
i=1
while i <= fleet_size:
    fleet_size_set.append(str(i))
    i += 1

# add 1-dimensional set 'm' with explanatory text 'bus index' to the GamsDatabase
m = db.add_set("m", 1, "bus index")
for i in fleet_size_set:
    m.add_record(i)

# add parameter 'batt' with domain 'm' with explanatory text 'battery capacity of bus bbb' to the GamsDatabase
batt = db.add_parameter_dc("batt", [m], "battery capacity of bus bbb")
for i in fleet_size_set:
    batt.add_record(i).value = chrom_batt[int(i)-1]

# add parameter to define the trips that each bus serves
bust = GamsParameter(db, "bust", 2, "whether a trip is served by a bus")
for k, v in dict_trip_set.items():
    bust.add_record(k).value = v

# add parameter to define the first trip of each bus
busf = GamsParameter(db, "busf", 2, "whether a trip is the first trip of a bus")
for k, v in dict_first_trip.items():
    busf.add_record(k).value = v

# add parameter to define the last trip of each bus
busl = GamsParameter(db, "busl", 2, "whether a trip is the last trip of a bus")
for k, v in dict_last_trip.items():
    busl.add_record(k).value = v

# add parameter to define the two adjacent trips of each bus
busaj = GamsParameter(db, "busaj", 3, "whether two trips are adjacent trips of a bus")
for k, v in dict_adjacent_trips.items():
    busaj.add_record(k).value = v

# add parameter to define the available charging time periods before serving a trip of each bus
busct = GamsParameter(db, "busct", 4, "available charging time periods before serving a trip of a bus")
for k, v in dict_available_charging_time.items():
    busct.add_record(k).value = v

# add parameter to define the available charging time periods at the depot for each bus
busctd = GamsParameter(db, "busctd", 2, "available charging time periods at the depot of a bus")
for k, v in dict_available_charging_time_d.items():
    busctd.add_record(k).value = v

# add parameter to define the start terminal of each trip
tripst = GamsParameter(db, "tripst", 2, "start terminal of a trip")
for k, v in dict_start_terminal.items():
    tripst.add_record(k).value = v

# add parameter to define the end terminal of each trip
tripet = GamsParameter(db, "tripet", 2, "end terminal of a trip")
for k, v in dict_end_terminal.items():
    tripet.add_record(k).value = v

# add parameter to define the energy need for serving a trip of each bus
busen = GamsParameter(db, "busen", 2, "energy need for serving a trip of a bus")
for k, v in dict_energy_trip.items():
    busen.add_record(k).value = v

# add parameter to define the energy consumption between two adjacent trips of each bus
busea = GamsParameter(db, "busea", 3, "energy consumption between two adjacent trips of a bus")
for k, v in dict_energy_between_trips.items():
    busea.add_record(k).value = v

# add parameter to define the energy consumption from the depot to the first trip of each bus
busdf = GamsParameter(db, "busdf", 2, "energy consumption from the depot to the first trip of a bus")
for k, v in dict_energy_depot_to_first_trip.items():
    busdf.add_record(k).value = v

# add parameter to define the energy consumption from the last trip to the depot of each bus
busld = GamsParameter(db, "busld", 2, "energy consumption from the last trip to the depot of a bus")
for k, v in dict_energy_last_trip_to_depot.items():
    busld.add_record(k).value = v

# add parameter to define the operating time of each bus
busst = GamsParameter(db, "busst", 1, "operating time of each bus")
for k, v in dict_operating_time.items():
    busst.add_record(k).value = v

# add parameter to define whether a terminal is equipped with a charging station
terch = GamsParameter(db, "terch", 1, "whether a terminal is equipped with a charging station")
for k, v in dict_term_charging.items():
    terch.add_record(k).value = v

# export the GamsDatabase to a GDX file with name 'data.gdx' located in the 'working_directory' of the GamsWorkspace
db.export("/Users/yihe/Documents/Gams_Test/data.gdx")